In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json


conf = SparkConf()
# conf.set("spark.app.name", "George Krup RDD app")

# sc = SparkContext(conf=conf)
# sc

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("lab01_gkrup")
         .getOrCreate())

In [3]:
# Файл u.data содержит все оценки, а файл u.item — список всех фильмов.
# film id for calculations == 96

In [4]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [5]:
spark

In [17]:
sc = spark.sparkContext

In [7]:
# !hdfs dfs -cat /labs/laba01/ml-100k/u.item

In [11]:
!hdfs dfs -cat /labs/laba01/ml-100k/README

SUMMARY & USAGE LICENSE

MovieLens data sets were collected by the GroupLens Research Project
at the University of Minnesota.
 
This data set consists of:
	* 100,000 ratings (1-5) from 943 users on 1682 movies. 
	* Each user has rated at least 20 movies. 
        * Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens web site
(movielens.umn.edu) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set. Detailed descriptions of
the data file can be found at the end of this file.

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under th

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType, DateType

In [8]:
item_schema = StructType(fields=[
    StructField("movie_id", IntegerType()),
    StructField("movie_title", StringType()),
    StructField("release_date", StringType()),
    StructField("video_release_date", StringType()),
    StructField("IMDb_URL", StringType()),
    StructField("unknown", IntegerType()),
    StructField("Action", IntegerType()),
    StructField("Adventure", IntegerType()),
    StructField("Animation", IntegerType()),
    StructField("Children's", IntegerType()),
    StructField("Comedy", IntegerType()),
    StructField("Crime", IntegerType()),
    StructField("Documentary", IntegerType()),
    StructField("Drama", IntegerType()),
    StructField("Fantasy", IntegerType()),
    StructField("Film-Noir", IntegerType()),
    StructField("Horror", IntegerType()),
    StructField("Musical", IntegerType()),
    StructField("Mystery", IntegerType()),
    StructField("Romance", IntegerType()),
    StructField("Sci-Fi", IntegerType()),
    StructField("Thriller", IntegerType()),
    StructField("War", IntegerType()),
    StructField("Western", IntegerType())
    
])


data_schema = StructType(fields=[
    StructField("user_id", IntegerType()),
    StructField("item_id", IntegerType()),
    StructField("rating", IntegerType()),
    StructField("timestamp", StringType())
])

In [9]:
item_df = spark.read\
          .schema(item_schema)\
          .format("csv")\
          .option("sep", "|")\
          .load("/labs/laba01/ml-100k/u.item")

In [10]:
data_df = spark.read\
          .schema(data_schema)\
          .format("csv")\
          .option("sep", "\t")\
          .load("/labs/laba01/ml-100k/u.data")

In [31]:
# сделаем через RDD
data_rdd_txt = sc.textFile('/labs/laba01/ml-100k/u.data')

In [34]:
data_rdd = data_rdd_txt.map(lambda x: (x.split()[1], x.split()[2])).cache()

In [41]:
my_movie = data_rdd.filter(lambda x: x[0] == '96').map(lambda x: (x[1], 1)).sortByKey(True).countByKey().items()
my_movie

dict_items([('1', 6), ('2', 20), ('3', 43), ('4', 123), ('5', 103)])

In [42]:
all_movie = data_rdd.map(lambda x: (x[1], 1)).sortByKey(True).countByKey().items() 
all_movie

dict_items([('1', 6110), ('2', 11370), ('3', 27145), ('4', 34174), ('5', 21201)])

In [43]:
result_json = {
                "hist_film": [b for a, b in my_movie],
                "hist_all": [b for a, b in all_movie]
              }

In [44]:
import json
with open('lab01.json', 'w') as f:
    json.dump(result_json, f)

In [45]:
spark.stop()